# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [2]:
#Criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
nav = webdriver.Chrome()


#importar/visualizar base de dados

df_produtos = pd.read_excel("buscas.xlsx")
df_produtos



,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 14 128gb,mini watch,1000,5000
1,rtx 3060,zota galax,1000,5000


In [11]:
#Entrar no google shop
#XrAfOe
produto = 'iphone 14 128gb'
termo_banido = 'mini watch'
preco_min = 2000
preco_max = 5000


#Para cada item dentro da nossa base de dados, para cada produto
#Procurar esse produto no google shopping



#verficar se algum dos produtos do google shopping esta dentro da faixa de preço
#procurar esse produto no buscape

#Salvar ofertas boas em um dataframe
#exporar para o excel
#Enviar pro e-mail o resultado da tabela

In [3]:

def pesquisa_preco_google(nav,produto,termo_banido,preco_min,preco_max):
    produto = produto.lower()
    termo_banido.lower()
    lista_nome_prod = produto.split(" ")
    lista_termos_banido = termo_banido.split(" ")
    nav.get("https://shopping.google.com.br/")
    nav.find_element(By.XPATH,'//*[@id="REsRA"]').send_keys(produto,Keys.ENTER)
    lista_resultado = nav.find_elements(By.CLASS_NAME,'i0X6df')
    lista = []
    for i,resultado in enumerate(lista_resultado):
        nome = resultado.find_element(By.CLASS_NAME,"tAxDx").text.lower()
        nome = nome.lower()
        tem_termo_ban = False
        for palavra in lista_termos_banido:
            if palavra in nome:
                tem_termo_ban=True

        tem_termo_produto = True
        for palavra in lista_nome_prod:
            if palavra not in nome:
                tem_termo_produto = False
        if not tem_termo_ban and tem_termo_produto:
            #if tem_termo_ban == False and tem_termo_produto ==True: -> mesma coisa do anterior
            preco = resultado.find_element(By.CLASS_NAME,"a8Pemb").text
            preco = preco.replace(" ","").replace("R$","").replace(".","").replace(",",".")
            try:
                preco = float(preco)
                preco_min = float(preco_min)
                preco_max = float(preco_max)
                if preco_min <= preco <= preco_max:
                    elemento_ref = resultado.find_element(By.CLASS_NAME,'bONr3b')
                    elemento_pai = elemento_ref.find_element(By.XPATH,'..')
                    link = elemento_pai.get_attribute("href")
                    lista.append([nome,preco,link])
            except:
                pass

    lista = pd.DataFrame.from_records(data=lista,columns=["produto","valor","Link"])
    return lista

In [4]:
def pesquisa_preco_buscape(nav,produto,termo_banido,preco_min,preco_max):
    lista = []
    produto = produto.lower()
    termo_banido.lower()
    lista_nome_prod = produto.split(" ")
    lista_termos_banido = termo_banido.split(" ")
    nav.get("https://www.buscape.com.br/")
    nav.find_element(By.XPATH,'//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto,Keys.ENTER)
    lista_elementos = nav.find_elements(By.CLASS_NAME,'ProductCard_ProductCard_Inner__gapsh')

    for resultado in lista_elementos:
        nome = resultado.find_element(By.CLASS_NAME,'ProductCard_ProductCard_Name__U_mUQ').text
        preco = resultado.find_element(By.CLASS_NAME,'Text_MobileHeadingS__HEz7L').text
        nome = nome.lower()
        tem_termo_ban = False
        for palavra in lista_termos_banido:
            if palavra in nome:
                tem_termo_ban=True
            tem_termo_produto = True
        for palavra in lista_nome_prod:
            if palavra not in nome:
                tem_termo_produto = False
        if not tem_termo_ban and tem_termo_produto:
            preco = preco.replace(" ","").replace("R$","").replace(".","").replace(",",".")    
            try:
                preco = float(preco)
                preco_min = float(preco_min)
                preco_max = float(preco_max)
                if preco_min <= preco <= preco_max:  
                    link = resultado.get_attribute('href')              
                    lista.append([nome,preco,link])
            except:
                pass
    lista = pd.DataFrame.from_records(data=lista,columns=["produto","valor","Link"])
    return lista


In [22]:
dic_shopping={}
for nome,termos,preco_min,preco_max in df_produtos.values:
    dic_shopping[nome] = pesquisa_preco_google(nav,produto=nome,termo_banido=termos,preco_min=preco_min,preco_max=preco_max)

In [35]:
df_pesquisa  = pd.DataFrame()
for nome,termos,preco_min,preco_max in df_produtos.values:
    busca = pesquisa_preco_buscape(nav,produto=nome,termo_banido=termos,preco_min=preco_min,preco_max=preco_max)
    try:
        busca = pd.DataFrame(busca)
        busca["origem"] = "Google"
        df_pesquisa = pd.concat([df_pesquisa,busca])
    except:
        pass
    busca = pesquisa_preco_google(nav,produto=nome,termo_banido=termos,preco_min=preco_min,preco_max=preco_max)
    try:
        busca = pd.DataFrame(busca)
        busca["origem"] = "Google"
        df_pesquisa = pd.concat([df_pesquisa,busca])
    except:
        pass

In [42]:
df_pesquisa

,produto,valor,Link,origem
0,smartphone apple iphone 14 128gb câmera dupla,3895.99,https://www.buscape.com.br/celular/smartphone-...,Google
1,smartphone apple iphone 14 plus 128gb câmera d...,4199.0,https://www.buscape.com.br/celular/smartphone-...,Google
2,smartphone apple iphone 14 plus vermelho 128gb...,4199.0,https://www.buscape.com.br/celular/smartphone-...,Google
3,smartphone apple iphone 14 vermelho 128gb câme...,3949.0,https://www.buscape.com.br/celular/smartphone-...,Google
0,apple iphone 14 - 128gb - meia-noite.,3879.0,https://www.google.com.br/url?url=https://www....,Google
...,...,...,...,...
35,pc gamer amd ryzen 5 4500 16gb (rtx 3060 12gb)...,3939.89,https://www.google.com.br/url?url=https://www....,Google
36,geforce | desktops | placa de video rtx 3060 t...,2465.0,https://www.google.com.br/url?url=https://www....,Google
37,pc gamer terabyte fury intel core i5 12400 / n...,4565.38,https://www.google.com.br/url?url=https://www....,Google
38,placa de video colorful igame geforce rtx 3060...,3259.0,https://www.google.com.br/url?url=https://www....,Google
